# Automatic differentiation



**! Caution with the functions np.sort, np.where, np.stack, np.broadcast_to !**
* Problem : the arguments are silently cast to np.ndarray, loosing autodiff information.
* Solution : use similarly named replacements from the ad library, which also apply to np.ndarray.

**! Caution with numpy scalars and array scalars !**
* Problem. In an expression 'a+b' where the l.h.s is a numpy scalar, and the r.h.s an array scalar of autodiff type, the r.h.s is silently cast loosing autodiff information.
* Solution : apply 'toarray' from ad library to a in that case (see below)

In [1]:
import sys; sys.path.append("..")
import numpy as np

In [2]:
import NumericalSchemes.AutomaticDifferentiation as ad

In [3]:
def reload_packages():
    import importlib
    ad = importlib.reload(sys.modules['NumericalSchemes.AutomaticDifferentiation'])
    ad.Sparse = importlib.reload(ad.Sparse)

# Sparse automatic differentiation

The sparse automatic differentiation class inherits from np.ndarray.

In [4]:
x = np.arange(5)
y_ad = 5-x+ad.Sparse.identity(x.shape)

Elementary properties are inherited

In [5]:
y_ad.ndim,y_ad.shape,y_ad.size,len(y_ad)

(1, (5,), 5, 5)

However, internal data includes coefficient and index arrays, with one additional dimension.

In [6]:
y_ad.size_ad

1

Left and right multiplication, addition, substraction, divition, work as usual.

In [7]:
x + 2*y_ad

spAD(array([10.,  9.,  8.,  7.,  6.]), array([[2.],
       [2.],
       [2.],
       [2.],
       [2.]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

A number of elementary mathematical functions are implemented.

In [8]:
np.sqrt(1+y_ad)

spAD(array([2.44948974, 2.23606798, 2.        , 1.73205081, 1.41421356]), array([[0.20412415],
       [0.2236068 ],
       [0.25      ],
       [0.28867513],
       [0.35355339]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

In [9]:
np.abs(y_ad-2)

spAD(array([3., 2., 1., 0., 1.]), array([[ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [-1.]]), array([[0],
       [1],
       [2],
       [3],
       [4]]))

Comparison operators return an ndarray, as well as integer valued functions.

In [10]:
y_ad <= (-1+x)

array([False, False, False,  True,  True])

In [11]:
np.floor(y_ad+0.5)

array([5., 4., 3., 2., 1.])

A base class ndarray can be recovered by requesting a view, or field 'value', or casting to np.array.

In [12]:
y_ad.view(np.ndarray), y_ad.value, np.array(y_ad)

(array([5., 4., 3., 2., 1.]),
 array([5., 4., 3., 2., 1.]),
 array([5., 4., 3., 2., 1.]))

maximum and minimum work as well

In [13]:
np.maximum(x,y_ad)

spAD(array([5., 4., 3., 3., 4.]), array([[1.],
       [1.],
       [1.],
       [0.],
       [0.]]), array([[0],
       [1],
       [2],
       [0],
       [0]]))

### Caveats

**! Caution with the np.sort function (does nothing) !**
Calling it on a spAD type does nothing. Use the spad.sort function instead.

In [15]:
ad.sort(y_ad) # Correct
# np.sort(ad) # Viciously, does nothing

spAD(array([1., 2., 3., 4., 5.]), array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]]), array([[4],
       [3],
       [2],
       [1],
       [0]]))

**! Caution with np.where, np.stack, np.broadcast_to (silently cast to base class) !**
Some other functions numpy functions will cast their arguments to the base class.
, are not numpy universal functions, and their variant from the spad library needs to be called.

In [19]:
ad.where(y_ad<x,y_ad,x) # Correct
#np.where(ad<arr,ad,arr) # casts to ndarray

spAD(array([0., 1., 2., 2., 1.]), array([[0.],
       [0.],
       [0.],
       [1.],
       [1.]]), array([[0],
       [0],
       [0],
       [3],
       [4]]))

In [23]:
a,b = ad.Sparse.identity((2,2))
ad.stack((a,b))
#np.stack((a,b)) # casts to ndarray

spAD(array([[0., 0.],
       [0., 0.]]), array([[[1.],
        [1.]],

       [[1.],
        [1.]]]), array([[[0],
        [1]],

       [[2],
        [3]]]))

**! Caution with numpy scalars and autodiff array scalars !**

In [25]:
reload_packages()
a=ad.Sparse.identity((2,))
b=np.ones((2,))

#Best
b=ad.toarray(b,type(a))
print("Correct (Best ?) ",b[0]-a[0])

# Other possibilities
b=np.ones((2,))
print("Correct",-(a[0]-b[0]))
print("Correct",ad.toarray(b[0])-a[0])
print("Correct",ad.toarray(b[0])-ad.toarray(a[0]))
print("Correct", ad.Sparse.spAD(b[0])-a[0])
print("Correct (different shape) ",b[[0]]-a[[0]])
print("Incorrect ! ",b[0]-a[0])

Correct (Best ?)  spAD(array(1.), array([-1.]), array([0]))
Correct spAD(array(1.), array([-1.]), array([0]))
Correct spAD(array(1.), array([-1.]), array([0]))
Correct spAD(array(1.), array([-1.]), array([0]))
Correct spAD(array(1.), array([-1.]), array([0]))
Correct (different shape)  spAD(array([1.]), array([[-1.]]), array([[0]]))
Incorrect !  1.0


**Non-silent failures:** np.reshape.

### Tested and working
np.take_along_axis, max, min, sum

In [27]:
y_ad.max(axis=0)

spAD(array(5.), array([1.]), array([0]))

In [28]:
np.min(y_ad,axis=0)

spAD(array(1.), array([1.]), array([4]))

In [29]:
np.sum(y_ad,axis=0)

spAD(array(15.), array([1., 1., 1., 1., 1.]), array([0, 1, 2, 3, 4]))